In [17]:
import numpy as np
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import requests 
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
!pip install geocoder
import geocoder

In [7]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')
neighborhoodList = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [8]:
kl_df = pd.DataFrame({"Neighborhood": neighborhoodList})

kl_df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [9]:
kl_df.shape

(70, 1)

In [18]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [19]:
coords = [ get_latlng(neighborhood) for neighborhood in kl_df["Neighborhood"].tolist() ]

In [20]:
coords

[[3.0576900000000364, 101.74388000000005],
 [3.1531525006886456, 101.70041313127312],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005],
 [3.08280000000002, 101.72281000000004],
 [3.1292000000000257, 101.67844000000008],
 [3.1347800000000348, 101.67262000000005],
 [3.111020000000053, 101.66283000000004],
 [3.09898000000004, 101.73499000000004],
 [3.1357600000000616, 101.70837000000006],
 [3.12916000000007, 101.68406000000004],
 [3.147770000000037, 101.70855000000006],
 [3.057800000000043, 101.68965000000009],
 [3.1434800000000678, 101.64433000000008],
 [3.152017197420035, 101.70102760046613],
 [3.129290000000026, 101.69892000000004],
 [3.17381000000006, 101.68276000000009],
 [3.061870000000056, 101.74675000000008],
 [3.163590000000056, 101.69811000000004],
 [3.147980000000075, 101.66798000000006],
 [3.1387586696676304, 101.6840455304707],
 [3.1387586696676304, 101.6840455304707],
 [3.156685175474611, 101.69807

In [21]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
kl_df['Latitude'] = df_coords['Latitude']
kl_df['Longitude'] = df_coords['Longitude']
print(kl_df.shape)
kl_df

(70, 3)


,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.153153,101.700413
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610
5,Bandar Tun Razak,3.082800,101.722810
6,Bangsar,3.129200,101.678440
7,Bangsar Park,3.134780,101.672620
8,Bangsar South,3.111020,101.662830
9,Batu 11 Cheras,3.098980,101.734990


In [23]:
kl_df.to_csv("kl_df.csv", index=False)

In [24]:
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516636, 101.6943028.


In [25]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [27]:
CLIENT_ID = '0K4NRFNTAJ1HSNKF1F45JEBIHBATKEKQAQXO45EGNUP3NHDM' 
CLIENT_SECRET = 'YISWHHNQCKWUKBNWXF53K4CWZQ3WA552VLCOF2M0M2S4IDUH' 
VERSION = '20180605' 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 0K4NRFNTAJ1HSNKF1F45JEBIHBATKEKQAQXO45EGNUP3NHDM
CLIENT_SECRET:YISWHHNQCKWUKBNWXF53K4CWZQ3WA552VLCOF2M0M2S4IDUH


In [28]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(kl_df['Latitude'], kl_df['Longitude'], kl_df['Neighborhood']):
    
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [29]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(6989, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
2,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
3,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
4,Alam Damai,3.05769,101.74388,Restoran Ikbal,3.061134,101.750220,Restaurant


In [30]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


In [31]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 307 uniques categories.


In [32]:
venues_df['VenueCategory'].unique()[:50]

array(['Supplement Shop', 'Noodle House', 'Seafood Restaurant',
       'Chinese Restaurant', 'Restaurant',
       'Vegetarian / Vegan Restaurant', 'Breakfast Spot', 'Food Court',
       'Asian Restaurant', 'Park', 'Other Great Outdoors',
       'Dim Sum Restaurant', 'Indian Restaurant', 'Snack Place', 'Spa',
       'Food Truck', 'Bubble Tea Shop', 'Convenience Store',
       'Chinese Breakfast Place', 'Japanese Restaurant', 'Pet Store',
       'Outlet Store', 'Dessert Shop', 'Café', 'Farmers Market',
       'Malay Restaurant', 'Cantonese Restaurant', 'Gym / Fitness Center',
       'Fast Food Restaurant', 'Steakhouse', 'Badminton Court', 'Bakery',
       'Hakka Restaurant', 'Athletics & Sports',
       'Middle Eastern Restaurant', 'Mamak Restaurant', 'Winery',
       'Burger Joint', 'College Bookstore', 'Grocery Store',
       'Halal Restaurant', 'Diner', 'Mexican Restaurant',
       'Monument / Landmark', 'Hostel', 'Latin American Restaurant',
       'Hotel', 'Hotel Pool', 'South India

In [33]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [34]:
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6989, 308)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Alam Damai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Alam Damai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Alam Damai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Alam Damai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Alam Damai,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(70, 308)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo
0,Alam Damai,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.060000,...,0.000000,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.00
1,"Ampang, Kuala Lumpur",0.00,0.00,0.00,0.00,0.000000,0.01,0.00,0.00,0.000000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00
2,Bandar Menjalara,0.00,0.00,0.01,0.00,0.000000,0.00,0.00,0.00,0.020000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,Bandar Sri Permaisuri,0.01,0.00,0.00,0.00,0.000000,0.01,0.00,0.00,0.030000,...,0.000000,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,Bandar Tasik Selatan,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.090000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5,Bandar Tun Razak,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.01,0.080000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,Bangsar,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.02,0.010000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00
7,Bangsar Park,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.02,0.000000,...,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.01,0.00
8,Bangsar South,0.01,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.010000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
9,Batu 11 Cheras,0.00,0.00,0.00,0.00,0.000000,0.00,0.00,0.00,0.060000,...,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [36]:
len(kl_grouped[kl_grouped["Shopping Mall"] > 0])

39

In [37]:
kl_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
kl_mall.head()

,Neighborhoods,Shopping Mall
0,Alam Damai,0.00
1,"Ampang, Kuala Lumpur",0.03
2,Bandar Menjalara,0.01
3,Bandar Sri Permaisuri,0.00
4,Bandar Tasik Selatan,0.01


In [39]:
kclusters = 3

kl_clustering = kl_mall.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

kmeans.labels_[0:10] 

array([0, 1, 2, 0, 2, 0, 1, 1, 2, 0], dtype=int32)

In [40]:
kl_merged = kl_mall.copy()

kl_merged["Cluster Labels"] = kmeans.labels_

In [41]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()


,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,0
1,"Ampang, Kuala Lumpur",0.03,1
2,Bandar Menjalara,0.01,2
3,Bandar Sri Permaisuri,0.00,0
4,Bandar Tasik Selatan,0.01,2


In [42]:
kl_merged = kl_merged.join(kl_df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head()

(70, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.03,1,3.153153,101.700413
2,Bandar Menjalara,0.01,2,3.190350,101.625450
3,Bandar Sri Permaisuri,0.00,0,3.103910,101.712260
4,Bandar Tasik Selatan,0.01,2,3.072750,101.714610


In [43]:
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(70, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.000000,0,3.057690,101.743880
29,Jinjang,0.000000,0,3.209500,101.658740
33,Kepong,0.000000,0,3.217500,101.637630
36,Maluri,0.000000,0,3.147890,101.694050
38,Miharja,0.000000,0,3.147890,101.694050
40,Pantai Dalam,0.000000,0,3.094760,101.667470
43,Salak South,0.000000,0,3.081020,101.697240
45,Semarak,0.000000,0,3.179916,101.721437
46,Sentul Raya,0.000000,0,3.187431,101.691453
47,Setapak,0.000000,0,3.188160,101.704150


In [44]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [45]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.0,0,3.057690,101.743880
29,Jinjang,0.0,0,3.209500,101.658740
33,Kepong,0.0,0,3.217500,101.637630
36,Maluri,0.0,0,3.147890,101.694050
38,Miharja,0.0,0,3.147890,101.694050
40,Pantai Dalam,0.0,0,3.094760,101.667470
43,Salak South,0.0,0,3.081020,101.697240
45,Semarak,0.0,0,3.179916,101.721437
46,Sentul Raya,0.0,0,3.187431,101.691453
47,Setapak,0.0,0,3.188160,101.704150


In [46]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
11,Brickfields,0.03,1,3.129160,101.684060
23,Dang Wangi,0.03,1,3.156685,101.698077
15,Bukit Nanas,0.03,1,3.152017,101.701028
41,"Pudu, Kuala Lumpur",0.04,1,3.133540,101.713070
39,Mont Kiara,0.03,1,3.165320,101.652430
7,Bangsar Park,0.05,1,3.134780,101.672620
35,Lembah Pantai,0.04,1,3.121202,101.663899
30,KL Eco City,0.03,1,3.117140,101.673890
65,Taman Tun Dr Ismail,0.03,1,3.152830,101.622710
66,Taman U-Thant,0.04,1,3.157700,101.724520


In [47]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
52,Sungai Besi,0.010000,2,3.050640,101.706130
4,Bandar Tasik Selatan,0.010000,2,3.072750,101.714610
8,Bangsar South,0.020000,2,3.111020,101.662830
63,Taman Sri Sinar,0.010417,2,3.190070,101.652930
56,Taman Desa,0.010000,2,3.102970,101.684710
10,"Batu, Kuala Lumpur",0.020000,2,3.135760,101.708370
2,Bandar Menjalara,0.010000,2,3.190350,101.625450
12,Bukit Bintang,0.020000,2,3.147770,101.708550
25,"Federal Hill, Kuala Lumpur",0.010000,2,3.136370,101.685640
49,Shamelin,0.020000,2,3.124580,101.735970
